In [ ]:
# Import necessary libraries
from openprompt.prompts import PtuningTemplate
from openprompt.plms import MLMTokenizerWrapper
from datasets import load_from_disk
from openprompt.prompts import ManualVerbalizer
from openprompt import PromptForClassification
from openprompt.data_utils import InputExample
from random import shuffle
from transformers import RobertaTokenizer, RobertaForMaskedLM
from openprompt.pipeline_base import PromptDataLoader
from transformers.optimization import get_linear_schedule_with_warmup
from transformers import AdamW
import torch
from datasets import load_from_disk
from tqdm import tqdm
import json
from openprompt.data_utils import FewShotSampler

# Load dataset
dataset_path = "/path/to/your/data/set"
raw_dataset = load_from_disk(dataset_path)

# Map textual labels to numeric labels
label_map = {"positive": 0, "negative": 1}

# Prepare datasets
dataset = {}
for split in ['train', 'validation']:
    dataset[split] = []
    # Shuffle and limit the dataset for efficiency
    raw_dataset[split] = raw_dataset[split].shuffle(seed=42).select(range(1000))
    for idx, data in enumerate(raw_dataset[split]):
        label_text = data["targets_pretokenized"].strip().lower()  # Extract label text
        label_numeric = label_map.get(label_text, -1)  # Convert to numeric label
        input_example = InputExample(text_a=data['inputs_pretokenized'], guid=idx, label=label_numeric)
        dataset[split].append(input_example)
print(dataset['train'][0])
print(type(dataset['train'][0]))

# Few-shot sampling from the training data
sampler = FewShotSampler(num_examples_per_label=30)
fewshot_data = sampler(dataset['train'], seed=42)

# Load RoBERTa model and tokenizer
roberta_path = "/path/to/facebook_roberta-base"
model = RobertaForMaskedLM.from_pretrained(roberta_path)
tokenizer = RobertaTokenizer.from_pretrained(roberta_path)
tokenizer.pad_token = tokenizer.eos_token  # Ensure padding token is set

# Logging setup
log_file = "prefix_tuning_results_roberta.json"
results = []

# Define hyperparameter search ranges
learning_rates = [1e-4, 3e-5]  # Learning rates to test
num_soft_tokens = [10, 50, 100]  # Number of soft tokens for prefix tuning
warmup_steps = [10, 20, 25]  # Warm-up steps for learning rate scheduler

# Loop through hyperparameter combinations
for lr in learning_rates:
    for tokens in num_soft_tokens:
        for warmup in warmup_steps:
            print(f"Testing: LR={lr}, Soft Tokens={tokens}, Warm-Up Steps={warmup}")

            # Reload the model and tokenizer for each configuration
            model = RobertaForMaskedLM.from_pretrained(roberta_path)
            tokenizer = RobertaTokenizer.from_pretrained(roberta_path)
            tokenizer.pad_token = tokenizer.eos_token

            # Initialize the P-tuning template
            template = PtuningTemplate(
                model=model,
                tokenizer=tokenizer,
                text='{"placeholder":"text_a"} {"mask"}',  # Template text
                prompt_encoder_type="lstm",  # Use LSTM-based prefix encoder
            )

            # Define a manual verbalizer
            verbalizer = ManualVerbalizer(
                tokenizer=tokenizer,
                num_classes=2,  # Binary classification
                label_words=[["positive", "good", "excellent", "wonderful"], ["negative", "bad", "horrible", "terrible"]],
                classes=[0, 1]  # Class labels
            )

            # Initialize the prompt model
            prompt_model = PromptForClassification(
                plm=model,
                template=template,
                verbalizer=verbalizer,
                freeze_plm=True,  # Freeze the pre-trained RoBERTa model
            )

            # Create dataloaders for training and validation
            train_dataloader = PromptDataLoader(
                dataset=fewshot_data,
                template=template,
                tokenizer=tokenizer,
                tokenizer_wrapper_class=MLMTokenizerWrapper,
                max_seq_length=480, decoder_max_length=3,
                batch_size=5, shuffle=True, teacher_forcing=False, predict_eos_token=False,
                truncate_method="tail",
            )
            
            validation_dataloader = PromptDataLoader(
                dataset=dataset["validation"],
                template=template,
                tokenizer=tokenizer,
                tokenizer_wrapper_class=MLMTokenizerWrapper,
                decoder_max_length=3,
                batch_size=5, shuffle=False, teacher_forcing=False, predict_eos_token=False,
                truncate_method="tail",
            )

            # Prepare optimizer and unfreeze necessary parameters
            optimizer_grouped_parameters = []
            for name, param in prompt_model.named_parameters():
                if not param.requires_grad and param.dtype in [torch.float32, torch.float64]:
                    param.requires_grad = True  # Unfreeze necessary parameters
                optimizer_grouped_parameters.append({'params': param})

            # Initialize optimizer and loss function
            optimizer = AdamW(optimizer_grouped_parameters, lr=lr)
            loss_func = torch.nn.CrossEntropyLoss()

            # Initialize learning rate scheduler
            scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup, num_training_steps=1000)

            # Training loop
            num_epochs = 10
            gradient_accumulation_steps = 1  # Gradients will accumulate before the optimizer step

            prompt_model.train()  # Set model to training mode
            for epoch in range(num_epochs):
                print(f"Epoch {epoch + 1}/{num_epochs}")
                total_loss = 0
                pbar = tqdm(train_dataloader, desc="Training")  # Progress bar

                for step, inputs in enumerate(pbar):
                    logits = prompt_model(inputs)  # Get model predictions
                    labels = inputs['label']  # Ground-truth labels

                    loss = loss_func(logits, labels)  # Compute loss
                    loss.backward()  # Backpropagation
                    
                    if (step + 1) % gradient_accumulation_steps == 0:
                        optimizer.step()  # Optimizer step
                        scheduler.step()  # Update learning rate
                        optimizer.zero_grad()  # Reset gradients
                    
                    total_loss += loss.item()
                    pbar.set_postfix({"loss": total_loss / (step + 1)})

            # Define evaluation function
            def evaluate(prompt_model, dataloader):
                prompt_model.eval()  # Set model to evaluation mode
                total, correct = 0, 0
                with torch.no_grad():
                    for inputs in dataloader:
                        logits = prompt_model(inputs)
                        preds = torch.argmax(logits, dim=-1)  # Predicted class
                        labels = inputs['label']
                        total += len(labels)
                        correct += (preds == labels).sum().item()
                return correct / total  # Compute accuracy

            # Evaluate the model on the validation set
            val_accuracy = evaluate(prompt_model, validation_dataloader)
            print(f"Validation Accuracy after Epoch {epoch + 1}: {val_accuracy:.4f}")

            # Log results
            result = {
                "learning_rate": lr,
                "num_soft_tokens": tokens,
                "warmup_steps": warmup,
                "final_loss": total_loss / (10 * len(train_dataloader)),
                "accuracy": val_accuracy
            }
            results.append(result)

            # Save intermediate results
            with open(log_file, "w") as f:
                json.dump(results, f, indent=4)
            
print("Tuning complete. Results saved to", log_file)


# Overview of Prefix Tuning for Sentiment Classification

This code implements a **Ptuning approach** for sentiment classification using the OpenPrompt framework and a pre-trained RoBERTa model. The script explores the application of soft prompts (Ptuning) and evaluates various hyperparameter configurations to optimize performance in a **few-shot learning setting**.

---

## Key Features

1. **Dataset Preparation**:
   - Loads a dataset from disk and processes it into the OpenPrompt-compatible `InputExample` format.
   - Maps sentiment labels (`positive`, `negative`) to numeric values for classification.
   - Implements a **few-shot sampler** to create a small training set with 30 examples per label.

2. **Prefix-Tuning**:
   - Uses the `PtuningTemplate` from OpenPrompt to generate **soft prompts** encoded with an LSTM-based architecture.
   - Dynamically adds soft tokens to the input for prefix tuning without modifying the pre-trained model weights.

3. **Manual Verbalizer**:
   - Maps predictions from the model's masked language modeling (MLM) head to the appropriate sentiment labels (`positive` or `negative`).
   - Allows flexible and interpretable classification.

4. **Hyperparameter Tuning**:
   - Tests various combinations of:
     - **Learning rates**: `1e-4`, `3e-5`
     - **Number of soft tokens**: `10`, `50`, `100`
     - **Warm-up steps**: `10`, `20`, `25`
   - Automates experimentation to identify the best configuration for prefix-tuning.

5. **Training Loop**:
   - Optimizes the prefix-tuning parameters using the AdamW optimizer.
   - Tracks training loss and uses a learning rate scheduler for fine-grained control of learning dynamics.
   - Supports gradient accumulation for efficient training with small batch sizes.

6. **Evaluation**:
   - Computes accuracy on the validation set after each training epoch.
   - Provides a reusable evaluation function to compute performance metrics.

7. **Result Logging**:
   - Logs hyperparameter settings, training loss, and validation accuracy.
   - Saves results in a JSON file for later analysis.

---

## Workflow Overview

### 1. **Dataset Handling**
- Loads the dataset and prepares it for few-shot learning by sampling a small subset of examples.

### 2. **Model and Template Setup**
- Loads a pre-trained RoBERTa model and tokenizer.
- Configures a soft prompt with `PtuningTemplate` and defines a `ManualVerbalizer` to map model predictions to labels.

### 3. **Hyperparameter Search**
- Iterates through combinations of learning rates, soft token counts, and warm-up steps.
- Reloads the model and tokenizer for each configuration to ensure independence between experiments.

### 4. **Training**
- Trains the prefix-tuning parameters while keeping the base RoBERTa model frozen.
- Tracks and logs training loss for each batch.

### 5. **Validation**
- Evaluates the model's performance on the validation set after each epoch.
- Computes validation accuracy to assess the effectiveness of each hyperparameter configuration.

### 6. **Logging Results**
- Records the results for each hyperparameter configuration in a structured JSON file.

---

## Applications

This code is designed for **sentiment classification** tasks in low-resource settings. The prefix-tuning approach leverages the power of pre-trained language models while keeping the computational cost low by only optimizing soft prompt parameters. It can be adapted to other classification tasks with minimal changes.

---

## Benefits of Prefix Tuning

1. **Efficient Fine-Tuning**:
   - Optimizes only a small set of parameters (soft prompts) instead of the entire model.
   - Reduces memory usage and speeds up training.

2. **Few-Shot Learning**:
   - Achieves strong performance with minimal labeled data.
   - Suitable for scenarios where large labeled datasets are unavailable.

3. **Modular and Flexible**:
   - Easy to experiment with different templates, verbalizers, and hyperparameters.
   - Compatible with various pre-trained models.

---

## Conclusion

This implementation showcases how prefix tuning can be applied to sentiment classification tasks using OpenPrompt and RoBERTa. By automating hyperparameter tuning and supporting few-shot learning, the script provides a robust framework for experimenting with prompt-based learning approaches in NLP tasks.
